# Working with GPUs in OpenVINO™

This tutorial provides a high-level overview of working with Intel GPUs in OpenVINO. It shows users how to use Query Device to list system GPUs and check their properties, and it explains some of the key properties. It shows how to compile a model on GPU with performance hints and how to use multiple GPUs using MULTI or CUMULATIVE_THROUGHPUT. 

The tutorial shows example commands for benchmark_app that users can run to compare GPU performance in different configurations. It also provides code for a basic end-to-end application that compiles a model on GPU and uses it to run inference.

## Introduction

1. Background and context on how GPUs are used to speed up inference
2. Introduce OpenVINO’s ability to run inference with GPUs
3. How to configure OpenVINO to work with GPUs (link to Configuration for GPU with OpenVINO page)

## Checking GPUs with Query Device

1. List GPUs with ie.get_available_devices
2. Check properties with ie.get_property
3. Brief descriptions of key properties

In this section we will see how to list the available GPUs and check their properties. Some of the key properties will also be defined.

### List GPUs with core.get_available_devices


Firstly, in order to use GPUs, we must make sure our system is detecting them correctly.
Running the following cell should output a list of compatible OpenVINO devices, in which our Intel GPUs should appear.

In [1]:
from openvino.runtime import Core

core = Core()
core.available_devices

['CPU']

If the GPUs are installed correctly in the system and still don't appear in the list, we should follow the steps described [here](https://docs.openvino.ai/latest/openvino_docs_install_guides_configurations_for_intel_gpu.html) and try again. Once we have the GPUs working with OpenVINO we can proceed with the next sections.

### Check properties with core.get_property

Now, to get information and customize the behavior of our GPUs, we can use device properties. Devices in OpenVINO, such as CPUs and GPUs, have two types of properties: read-only and read-write. The former mainly shows information about the hardware itself like the device name or supported data types, while the latter allows us to tweak how the model is compiled, for instance to reduce latency or increase throughput.

So, to get the value of a property, such as the device name, we can use the `core.get_property` method as follows

In [2]:
core.get_property("CPU", "FULL_DEVICE_NAME")

'Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz'

Each device also has a specific property, called `SUPPORTED_PROPERTIES`, that allows us to see all the available properties in the device (including the `SUPPORTED_PROPERTIES` itself). To do this, we repeat the above command

In [3]:
core.get_property("CPU", "SUPPORTED_PROPERTIES")

{'SUPPORTED_PROPERTIES': 'RO',
 'AVAILABLE_DEVICES': 'RO',
 'RANGE_FOR_ASYNC_INFER_REQUESTS': 'RO',
 'RANGE_FOR_STREAMS': 'RO',
 'FULL_DEVICE_NAME': 'RO',
 'OPTIMIZATION_CAPABILITIES': 'RO',
 'CACHING_PROPERTIES': 'RO',
 'CACHE_DIR': 'RO',
 'NUM_STREAMS': 'RW',
 'AFFINITY': 'RW',
 'INFERENCE_NUM_THREADS': 'RW',
 'PERF_COUNT': 'RW',
 'INFERENCE_PRECISION_HINT': 'RW',
 'PERFORMANCE_HINT': 'RW',
 'PERFORMANCE_HINT_NUM_REQUESTS': 'RW'}

Note that the value for each property has either a "RO" or "RW", which corresponds to the two types mentioned previously, "**R**ead-**O**nly" and "**R**-**W**rite" respectively.

### Brief descriptions of key properties

Each device has several properties as seen in the last command. Some of the key properties would be

* Read-Only
    * `FULL_DEVICE_NAME` - The product name of the GPU and whether it is an integrated or discrete GPU (iGPU or dGPU).
    * `OPTIMIZATION_CAPABILITIES` - The model data types (INT8, FP16, FP32, etc) that are supported by this GPU.
    * `GPU_EXECUTION_UNITS_COUNT` - The execution cores available in the GPU's architecture, which is a relative measure of the GPU's processing power.
    * `RANGE_FOR_STREAMS` - The number of processing streams available on the GPU that can be used to execute parallel inference requests. When compiling a model in LATENCY or THROUGHPUT mode, OpenVINO will automatically select the best number of streams for low latency or high throughput.
    * `DEVICE_GOPS` - The Giga operations per second count (GFLOPS) for each precision the device supports.
* Read-Write
    * `PERFORMANCE_HINT` - A high-level way to tune the device for a specific performance metric, such as latency or throughput, without worrying about device-specific settings.
    * `INFERENCE_PRECISION_HINT` - A high-level way to specify which model data type to use for inference.

To sum up this section, we can check the value for each property by simply looping through the dictionary returned by `core.get_property("GPU", "SUPPORTED_PROPERTIES")` and then querying for that property.

In [11]:
supported_properties = core.get_property("CPU", "SUPPORTED_PROPERTIES")
indent = len(max(supported_properties, key=len))
for prop in supported_properties:
    print(f"{prop:>{indent}}:", core.get_property("CPU", prop))

          SUPPORTED_PROPERTIES: {'SUPPORTED_PROPERTIES': 'RO', 'AVAILABLE_DEVICES': 'RO', 'RANGE_FOR_ASYNC_INFER_REQUESTS': 'RO', 'RANGE_FOR_STREAMS': 'RO', 'FULL_DEVICE_NAME': 'RO', 'OPTIMIZATION_CAPABILITIES': 'RO', 'CACHING_PROPERTIES': 'RO', 'CACHE_DIR': 'RO', 'NUM_STREAMS': 'RW', 'AFFINITY': 'RW', 'INFERENCE_NUM_THREADS': 'RW', 'PERF_COUNT': 'RW', 'INFERENCE_PRECISION_HINT': 'RW', 'PERFORMANCE_HINT': 'RW', 'PERFORMANCE_HINT_NUM_REQUESTS': 'RW'}
             AVAILABLE_DEVICES: ['']
RANGE_FOR_ASYNC_INFER_REQUESTS: (1, 1, 1)
             RANGE_FOR_STREAMS: (1, 12)
              FULL_DEVICE_NAME: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
     OPTIMIZATION_CAPABILITIES: ['FP32', 'FP16', 'INT8', 'BIN', 'EXPORT_IMPORT']
            CACHING_PROPERTIES: {}
                     CACHE_DIR: 
                   NUM_STREAMS: 1
                      AFFINITY: Affinity.NONE
         INFERENCE_NUM_THREADS: 0
                    PERF_COUNT: False
      INFERENCE_PRECISION_HINT: <Type: 'float32'>
   

## Compiling a Model on GPU

1. Compile with default configuration (ie.compile_model(model, “GPU”)
2. Throughput and latency performance hints
3. Using multiple GPUs with multi-device and cumulative throughput

We now know what is a GPU, how to check if we have one and its properties but, how do we actually **use** one?

### Compile with default configuration (ie.compile_model(model, “GPU”)

In fact, due to the [AUTO plugin](https://docs.openvino.ai/latest/openvino_docs_OV_UG_supported_plugins_AUTO.html#how-auto-works), we might already be using GPUs if they are properly recognized by OpenVINO. Despite this, if we want to use a specific device, we can do so by compiling our models as follows

In [9]:
# load any model
model = core.read_model(model="../001-hello-world/model/v3-small_224_1.0_float.xml")

# compile the loaded model using the GPU device
compiled_model = core.compile_model(model, "CPU")

Note that above we are using `"GPU"`, which is an alias for `"GPU.0"` according to the [docs](https://docs.openvino.ai/latest/openvino_docs_OV_UG_supported_plugins_GPU.html). Actually, as expected, any of the values returned by `core.available_devices` are valid device specifiers.

Once we have a compiled model, we can check for its properties, just as we did with the GPU devices in the previous section. Instead of calling `core.get_property` with a specific device, we can call `compiled_model.get_property` directly with the property name.

In [10]:
supported_properties = compiled_model.get_property("SUPPORTED_PROPERTIES")
indent = len(max(supported_properties, key=len))
for prop in supported_properties:
    print(f"{prop:>{indent}}:", compiled_model.get_property(prop))

            SUPPORTED_PROPERTIES: {'SUPPORTED_PROPERTIES': 'RO', 'NETWORK_NAME': 'RO', 'OPTIMAL_NUMBER_OF_INFER_REQUESTS': 'RO', 'NUM_STREAMS': 'RO', 'AFFINITY': 'RO', 'INFERENCE_NUM_THREADS': 'RO', 'PERF_COUNT': 'RO', 'INFERENCE_PRECISION_HINT': 'RO', 'PERFORMANCE_HINT': 'RO', 'PERFORMANCE_HINT_NUM_REQUESTS': 'RO'}
                    NETWORK_NAME: v3-small_224_1.0_float
OPTIMAL_NUMBER_OF_INFER_REQUESTS: 1
                     NUM_STREAMS: 1
                        AFFINITY: Affinity.NONE
           INFERENCE_NUM_THREADS: 12
                      PERF_COUNT: False
        INFERENCE_PRECISION_HINT: <Type: 'float32'>
                PERFORMANCE_HINT: PerformanceMode.UNDEFINED
   PERFORMANCE_HINT_NUM_REQUESTS: 0


One of the key properties defined before that reappears here is the `PERFORMANCE_HINT`. The behavior of the hint itself is the same at both levels, except for the fact that the hint in the device can be considered as a "global" property, i.e all models created on that device will use the same value of the hint, whereas in the compiled model acts as a "local" one, i.e only affects that model. See [the docs](https://docs.openvino.ai/latest/openvino_docs_OV_UG_query_api.html#setting-properties-globally) for more info.

Until now we have only queried either the device or the compiled model for properties but, how can we modify those that are read-write?

### Throughput and latency performance hints

To modify properties at the device level, we can use the `core.set_property` method, in which instead of just using the property name, a dictionary is required specifying the value as well. For example, if we want to change the `PERFORMANCE_HINT` to improve latency we can do it like

In [25]:
core.set_property("CPU", {"PERFORMANCE_HINT": "LATENCY"})
core.get_property("CPU", "PERFORMANCE_HINT")

<PerformanceMode.LATENCY: 1>

If instead we only want to affect a certain model, we need to add the dictionary as the third argument in the `core.compile_model` method. For instance, to improve throughtput for just this model the following works

In [29]:
compiled_model = core.compile_model(model, "CPU", {"PERFORMANCE_HINT": "THROUGHPUT"})
compiled_model.get_property("PERFORMANCE_HINT")

<PerformanceMode.THROUGHPUT: 2>

##### Throughput

In [7]:
compiled_model = core.compile_model(model, "CPU", {"PERFORMANCE_HINT": "THROUGHPUT"})

In [8]:
compiled_model.get_property("PERFORMANCE_HINT")

<PerformanceMode.THROUGHPUT: 2>

##### Latency

In [9]:
compiled_model = core.compile_model(model, "CPU", {"PERFORMANCE_HINT": "LATENCY"})

In [10]:
compiled_model.get_property("PERFORMANCE_HINT")

<PerformanceMode.LATENCY: 1>

### Using multiple GPUs with multi-device and cumulative throughput

##### Multi-device

In [40]:
compiled_model = core.compile_model(model=model, device_name="MULTI:CPU")

In [41]:
compiled_model.get_property("PERFORMANCE_HINT")

'THROUGHPUT'

##### Cumulative Throughput

In [34]:
compiled_model = core.compile_model(model, "CPU", {"PERFORMANCE_HINT": "CUMULATIVE_THROUGHPUT"})

In [36]:
compiled_model.get_property("PERFORMANCE_HINT")

<PerformanceMode.THROUGHPUT: 2>

## Performance Comparison with benchmark_app

1. Commands showing users how to run benchmark_app on GPU with various performance hints
2. Show performance results with a basic model (person-detection-0303, perhaps)

For further details check https://docs.openvino.ai/latest/openvino_inference_engine_tools_benchmark_tool_README.html#benchmark-python-tool

### Commands showing users how to run benchmark_app on GPU with various performance hints

In [11]:
!benchmark_app -m notebooks/001-hello-world/model/v3-small_224_1.0_float.xml -hint latency -d GPU

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] Device info:
[ ERROR ] Device with "GPU" name is not registered in the OpenVINO Runtime
Traceback (most recent call last):
  File "/Users/juliomorero/Documents/bdti/openvino_env/lib/python3.10/site-packages/openvino/tools/benchmark/main.py", line 104, in main
    benchmark.print_version_info()
  File "/Users/juliomorero/Documents/bdti/openvino_env/lib/python3.10/site-packages/openvino/tools/benchmark/benchmark.py", line 48, in print_version_info
    for device, version in self.core.get_versions(self.device).items():
RuntimeError: Device with "GPU" name is not registered in the OpenVINO Runtime


In [ ]:
!benchmark_app -m notebooks/001-hello-world/model/v3-small_224_1.0_float.xml -hint latency

In [ ]:
!benchmark_app -m notebooks/001-hello-world/model/v3-small_224_1.0_float.xml -hint throughput

In [ ]:
!benchmark_app -m notebooks/001-hello-world/model/v3-small_224_1.0_float.xml -hint cumulative_throughput

### Show performance results with a basic model (person-detection-0303, perhaps)

## Basic Application Using GPUs

1. Provide end-to-end sample code for running inference on GPU in a basic application

## Conclusion

1. GPUs are easy to use with OpenVINO and considerably boost performance
2. Links to OpenVINO documentation where readers can learn more